 We have created a SQLite database containing all the English data from Wikipedia. This implementation <br>
 of the database has three tables Pages, Links, and Redirects. Using these tables, all of Wikipedia can be <br>
 thought of as a graph. <br><br>
 The below blog describes how the SQLlite database can be created on your own machine. <br>
 https://github.com/jwngr/sdow/blob/master/docs/data-source.md#get-the-data-yourself

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

import timeit

After generating our database, connection is very simple and follows:

In [2]:
database = "sdow.sqlite"
connection = sql.connect(database)

Let us make a couple of queries to each table. We can also time the efficiency for each query.

We start with the <b>Pages</b> table. This table holds the columns id, title, and is_redirect.

In [3]:
query_pages = '''SELECT * FROM pages where title = 'Yellow' '''

In [4]:
%%time
df = pd.read_sql_query(query_pages, connection)
df.head()

Wall time: 3.61 s


,id,title,is_redirect
0,34368,Yellow,0


Notice it takes a full 3 seconds to query the Pages table. This is because we searched using a title string <br>
instead of the id value.

Let us consider the actual size of the Pages table:

In [5]:
#Select all from pages, including redirects.
query_pages = '''SELECT * FROM pages '''

In [6]:
%%time
df = pd.read_sql_query(query_pages, connection)
df.head()

Wall time: 42.3 s


,id,title,is_redirect
0,10,AccessibleComputing,1
1,12,Anarchism,0
2,13,AfghanistanHistory,1
3,14,AfghanistanGeography,1
4,15,AfghanistanPeople,1


In [7]:
len(df)

14973896

We actually have 14973896 articles in our graph (including redirects).

Next, we try to query the <b>query_links</b> table. <br><br>
The Links table contains the columns id, outgoing_links_count, incoming_links_count, <br>
outgoing_links, and incoming_links.

In [8]:
query_links = '''SELECT * FROM links where id = 34368'''

In [9]:
%%time
df = pd.read_sql_query(query_links, connection)
df.head()

Wall time: 13 ms


,id,outgoing_links_count,incoming_links_count,outgoing_links,incoming_links
0,34368,646,1918,1001908|10025|100549|10134|1023|1055539|10568|...,10012055|1001908|10025|1003148|10040190|100536...


Notice that the Links table only takes about 4-10ms per query! This is much better then querying the Pages table. <b>Why?</b> <br>
Because we searched with the id!

Let us see how large this table actually is.

In [45]:
query_links = '''SELECT * FROM links'''

In [ ]:
%%time
df = pd.read_sql_query(query_links, connection)
df.head()

In [29]:
len(df)

1

Putting the entire links table into memory takes a whopping 35 minutes even though there are less than half <br>
of the number of article pages. Why does it take so long? <br><br>
This has to do with the number of incoming and outgoing links each article can have. Let us quickly <br>
take a look at the avergare number of in and out links.

In [30]:
q = '''SELECT AVG(outgoing_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head()

,AVG(outgoing_links_count)
0,81.224944


In [31]:
q = '''SELECT AVG(incoming_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head()

,AVG(incoming_links_count)
0,81.224944


In [32]:
q = '''SELECT MAX(outgoing_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head()

,MAX(outgoing_links_count)
0,10524


In [33]:
query_links = '''SELECT * FROM links where outgoing_links_count = '10524' '''
df = pd.read_sql_query(query_links, connection)
df.head() #Index_of_Singapore-related_articles

,id,outgoing_links_count,incoming_links_count,outgoing_links,incoming_links
0,2596980,10524,150,10009609|1002286|10027447|10033793|10040685|10...,1015816|1047876|1070098|10846179|10877172|1140...


In [34]:
q = '''SELECT MAX(incoming_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head()

,MAX(incoming_links_count)
0,1116378


In [35]:
query_links = '''SELECT * FROM links where incoming_links_count = '1116378' '''
df = pd.read_sql_query(query_links, connection)
df.head() #International_Standard_Book_Number

,id,outgoing_links_count,incoming_links_count,outgoing_links,incoming_links
0,14919,534,1116378,1003435|1015276|1015432|10179938|1032603|10367...,1000|1000005|1000006|1000007|100001|10000110|1...


In [36]:
q = '''SELECT MIN(outgoing_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head() 

,MIN(outgoing_links_count)
0,0


In [47]:
q = '''SELECT MIN(incoming_links_count) FROM links '''
df = pd.read_sql_query(q, connection)
df.head()

,MIN(incoming_links_count)
0,0


In [48]:
q = '''SELECT * FROM links where incoming_links_count = 0'''
df = pd.read_sql_query(q, connection)
df.head()

,id,outgoing_links_count,incoming_links_count,outgoing_links,incoming_links
0,1597,4,0,1601|160197|160210|783,
1,1607,3,0,13180183|200530|23787,
2,1608,3,0,13261274|2190669|23535646,
3,1663,4,0,1676|1679|69603|70643,
4,1689,7,0,1657|1677|1679|1686|2177584|69603|8397423,


In [49]:
len(df)

313101

In [50]:
q = '''SELECT * FROM links where outgoing_links_count = 0'''
df = pd.read_sql_query(q, connection)
df.head()

,id,outgoing_links_count,incoming_links_count,outgoing_links,incoming_links
0,55558,0,420,,10127924|1045028|1055471|1080344|10805630|1192...
1,60946,0,42,,10563365|1373695|1465527|1466174|17006|1729005...
2,273143,0,1,,33565271
3,352347,0,1,,352340
4,364864,0,3,,15810482|18492337|91963


In [51]:
len(df)

932

We find some interesting information <br>
The average number of links between any two nodes is about 81. <br> 
There exists an article that has over 10,000 in-links. This indicates "important" articles exist and draw a lot of references.<br>
Finally, there does not exist any pages that are completely disconnected - no pages have 0 incoming and 0 outgoing links. <br> <br>
Consider though that this does <b>not</b> guarantee that there is a path from one <br>
article to another since the graph is directed. For instance, if you started from an article with no out-links.

Lastly, we take a look at the Redirects table. <br>
This table only has source_id and target_id columns. This table is used as many searches in Wikipedia actually <br>
redirect to another article entirely. 

In [40]:
query_redirects = '''SELECT * FROM redirects where source_id = 10 '''

In [41]:
%%time
df = pd.read_sql_query(query_redirects, connection)
df.head()

Wall time: 2 ms


,source_id,target_id
0,10,411964


As we did before, let us check the table size.

In [42]:
query_redirects = '''SELECT * FROM redirects'''

In [43]:
%%time
df = pd.read_sql_query(query_redirects, connection)
df.head()

Wall time: 24.4 s


,source_id,target_id
0,10,411964
1,13,13813
2,14,12681
3,15,66468
4,18,6684


In [44]:
len(df)

8975917

Interestingly, 9 million of our 14 million articles are link redirects to another article!